In [ ]:
# Melakukan Import module numpy, pandas, dan seaborn
import numpy as np # Untuk perhitungan yang berkaitan dengan matematika
import pandas as pd # Untuk read csv dan processing data pada dataframe
import seaborn as sns # Untuk membuat grafik dari data yang telah diproses
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

import warnings
warnings.filterwarnings('ignore')

from scipy import stats # Untuk mempermudah perhitungan statistika
from datetime import * # Untuk memudahkan partisi pada time

# Memudahkan import csv dari input kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Melakukan import csv dari data-data yang disediakan dengan panda.read_csv

gd = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')

en_test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
en_train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')

cu_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
cu_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')

In [ ]:
gd.head() # Melihat secara sekilas data bulding_metadata.csv

In [ ]:
# Melihat informasi data frame bulding_metadata.csv,seperti banyaknya row dan column, type data setiap variabel
gd.info()

# Melihat jumlah missing values pada data frame bulding_metadata.csv
gd.isnull().sum()

In [ ]:
# Membuat fungsi untuk melakukan reduce pada data untuk meringankan program agar tetap dapat berjalan

def reduce_mem_usage(df_train):
    start_mem = df_train.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df_train.columns:
        col_type = df_train[col].dtype

        if col_type != object:
            c_min = df_train[col].min()
            c_max = df_train[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df_train[col] = df_train[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df_train[col] = df_train[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df_train[col] = df_train[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df_train[col] = df_train[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df_train[col] = df_train[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df_train[col] = df_train[col].astype(np.float32)
                else:
                    df_train[col] = df_train[col].astype(np.float64)
        else:
            df_train[col] = df_train[col].astype('category')

    end_mem = df_train.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df_train

PREPROCESSING DATA TEST

In [ ]:
# Melihat secara sekilas data pada test.csv
en_test.head()

In [ ]:
# Melihat informasi data frame test.csv,seperti banyaknya row dan column, type data setiap variabel
en_test.info()

# Melihat jumlah missing values pada data frame test.csv
en_test.isnull().sum()

In [ ]:
# Melakukan left join data test.csv dengan building_metadata.csv pada column building_id
test_en_gd = en_test.merge(gd, how = 'left', on = 'building_id')

# Melihat secara sekilas data pada gabungan/hasil left join data test.csv dengan building_metadata.csv
test_en_gd.head()

In [ ]:
# Melihat secara seklilas data pada weather_test.csv
cu_test.head()

In [ ]:
# Melihat informasi pada data frame weather_test.csv
cu_test.info()

# Melihat jumlah missing values pada data frame weather_test.csv
cu_test.isnull().sum()

In [ ]:
# Melakukan cek pola missing values yang terdapat pada cu_test

sns.heatmap(cu_test.isnull(), cbar=False)

In [ ]:
# Missing values dapat diisi dengan median dari suatu data pada site tertentu dan pada waktu tertentu
# Untuk memudahkan, waktu yang diambil adalah waktu hours setiap harinya
# Mengambil timestamp asli, sebelum dilakukan manipulasi
cu_test_timestamp = cu_test['timestamp']
cu_test_timestamp = cu_test_timestamp.to_frame()

In [ ]:
# Mengisi missing values pada data yang berhubungan dengan weather
# Mengambil hours dari timestamp untuk nanti mencari median dari weather pada hours dan pada site tertentu
# Memanipulasi timestamp dengan menggantinya menjadi hours
cu_test['timestamp'] = pd.to_datetime(cu_test['timestamp'])
cu_test['timestamp'] = pd.to_datetime(cu_test['timestamp'], format = "%Y-%m-%d %H:%M:%S").dt.strftime('%m,%H')
cu_test['timestamp'].tail()

# Program dibawah ini akan membuat column yang berisi median dari data-data pada jam dan site tertentu
# Nantinya column yang telah dibuat akan digunakan untuk mengisi missing values dengan median sesuai dengan variabelnya

cu_testa = cu_test.groupby(['site_id','timestamp'])['air_temperature'].median()
cu_testa = cu_testa.to_frame()
cu_test = cu_test.merge(cu_testa, how = 'left', on = ['timestamp','site_id'])
cu_test['air_temperature_x'] = cu_test['air_temperature_x'].fillna(cu_test['air_temperature_y'])

cu_testc = cu_test.groupby(['site_id','timestamp'])['cloud_coverage'].median()
cu_testc = cu_testc.to_frame()
cu_test = cu_test.merge(cu_testc, how = 'left', on = ['timestamp','site_id'])
cu_test['cloud_coverage_x'] = cu_test['cloud_coverage_x'].fillna(cu_test['cloud_coverage_y'])

cu_testd = cu_test.groupby(['site_id','timestamp'])['dew_temperature'].median()
cu_testd = cu_testd.to_frame()
cu_test = cu_test.merge(cu_testd, how = 'left', on = ['timestamp','site_id'])
cu_test['dew_temperature_x'] = cu_test['dew_temperature_x'].fillna(cu_test['dew_temperature_y'])

cu_testp = cu_test.groupby(['site_id','timestamp'])['precip_depth_1_hr'].median()
cu_testp = cu_testp.to_frame()
cu_test = cu_test.merge(cu_testp, how = 'left', on = ['timestamp','site_id'])
cu_test['precip_depth_1_hr_x'] = cu_test['precip_depth_1_hr_x'].fillna(cu_test['precip_depth_1_hr_y'])

cu_tests = cu_test.groupby(['site_id','timestamp'])['sea_level_pressure'].median()
cu_tests = cu_tests.to_frame()
cu_test = cu_test.merge(cu_tests, how = 'left', on = ['timestamp','site_id'])
cu_test['sea_level_pressure_x'] = cu_test['sea_level_pressure_x'].fillna(cu_test['sea_level_pressure_y'])

cu_testwd = cu_test.groupby(['site_id','timestamp'])['wind_direction'].median()
cu_testwd = cu_testwd.to_frame()
cu_test = cu_test.merge(cu_testwd, how = 'left', on = ['timestamp','site_id'])
cu_test['wind_direction_x'] = cu_test['wind_direction_x'].fillna(cu_test['wind_direction_y'])

cu_testws = cu_test.groupby(['site_id','timestamp'])['wind_speed'].median()
cu_testws = cu_testws.to_frame()
cu_test = cu_test.merge(cu_testws, how = 'left', on = ['timestamp','site_id'])
cu_test['wind_speed_x'] = cu_test['wind_speed_x'].fillna(cu_test['wind_speed_y'])

In [ ]:
# Karena timestamp masih berbentuk jam hasil manipulasi, lakukan join timestamp awal yang disimpan
cu_test = pd.merge(cu_test, cu_test_timestamp, left_index=True, right_index=True)

# Karena masih terdapat column yang berisi median pada setiap variabel, lakukan drop column tersebut
cu_test = cu_test.drop(columns = ['timestamp_x','air_temperature_y',
                                  'cloud_coverage_y','dew_temperature_y',
                                  'precip_depth_1_hr_y','sea_level_pressure_y',
                                  'wind_direction_y', 'wind_speed_y'])

# Mengurutkan column seperti awal dataframe dan melakukan rename column seperti awal
cu_test = cu_test[['site_id', 'timestamp_y', 'air_temperature_x','cloud_coverage_x', 'dew_temperature_x', 
                   'precip_depth_1_hr_x', 'sea_level_pressure_x','wind_direction_x', 'wind_speed_x']]
cu_test = cu_test.rename(columns={"timestamp_y": "timestamp","air_temperature_x": "air_temperature",
                        "cloud_coverage_x": "cloud_coverage","dew_temperature_x": "dew_temperature",
                        "precip_depth_1_hr_x": "precip_depth_1_hr","sea_level_pressure_x": "sea_level_pressure",
                        "wind_direction_x": "wind_direction","wind_speed_x": "wind_speed"})

In [ ]:
# Melakukan cek apakah missing values masih terdapat dalam data
cu_test.isnull().sum()

In [ ]:
# Melakukan cek pola missing values yang terdapat pada cu_train

sns.heatmap(cu_test.isnull(), cbar=False)

In [ ]:
# Missing values yang masih terdapat pada cloud_coverage dapat dilakukan fillna(0)
# Dengan asumsi bahwa missing values terjadi karena tidak adanya cloud pada waktu tersebut, sehingga dapat diisi 0
cu_test['cloud_coverage'] = cu_test['cloud_coverage'].fillna(0)

# Missing values yang masih terdapat pada precip_depth_1_hr dapat dilakukan fill dengan median dari data tersebut
median_precip = cu_test['precip_depth_1_hr'].median()
cu_test['precip_depth_1_hr'] = cu_test['precip_depth_1_hr'].fillna(median_precip)

# Missing values yang masih terdapat pada sea_level_pressure dapat dilakukan fill dengan median dari data tersebut
median_sea = cu_test['sea_level_pressure'].median()
cu_test['sea_level_pressure'] = cu_test['sea_level_pressure'].fillna(median_sea)

In [ ]:
# Melakukan cek persentase missing values pada data
cu_test.count()/len(cu_test)

In [ ]:
# Melakukan reduce untuk data test_en_gd dan cu_test
test_en_gd = reduce_mem_usage(test_en_gd)
cu_test = reduce_mem_usage(cu_test)

In [ ]:
# Melakukan left join data gabungan pada test_en_gd dengan data cuaca weather_test pada column timestamp dan site_id
df_test = test_en_gd.merge(cu_test, how = 'left', on = ['site_id','timestamp'])

# Melihat sekilas pada data frame df_test yang telah digabung dari 3 dataframe
df_test.head()

In [ ]:
# Melihat informasi data frame pada gabungan/hasil left join data 
df_test.info()

# Melihat persentase missing values pada data frame df_test
df_test.count()/len(df_test)

In [ ]:
# Mencari nilai yang menjadi missing values
df_test_null = df_test[df_test.air_temperature.isnull()]
df_test_null.head()

In [ ]:
# Missing values terjadi karena ketiadaan data pada suatu rentang waktu timestamp weather_test.csv
# Missing values pada tahap ini dapat dilakukan dropna() 
df_test = df_test.dropna(subset=['air_temperature']) 

In [ ]:
# Melakukan cek apakah missing values pada data yang berkaitan dengan weather telah terhapus
print(df_test.isnull().sum())

# Melakukan cek info data df_test
print(df_test.info())

In [ ]:
# Mengganti type data dari column timestamp dari object menjadi date time karena merupakan data time
df_test['timestamp'] = df_test['timestamp'].astype('object')
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

#Melihat informasi data frame untuk melihat apakah type data timestamp sudah terganti menjadi date time
df_test.info()

In [ ]:
# Melakukan cek sekilas kekonsistenan dari beberapa variabel yang memungkinkan terdapat ketidakkonsistenan

ts_met = df_test['meter'].unique()
ts_met.sort()
print(ts_met)

ts_siteid = df_test['site_id'].unique()
ts_siteid.sort()
print(ts_siteid)

ts_primaryu = df_test['primary_use'].unique()
print(ts_primaryu)

ts_yearb = df_test['year_built'].unique()
ts_yearb.sort()
print(ts_yearb)

Missing values pada year_built dan floor_count pada data df_test dapat dihiraukan, karena bukan merupakan poin penting dalam melakukan analisa data penggunaan energi

Untuk data test outlier dapat dibiarkan karena tidak akan dilakukan analisa statistik

In [ ]:
print('Data test yang telah dilakukan preprocessing:')
df_test

In [ ]:
df_test.to_csv('eda_kelompok_a_df_test.csv', index = False)